# Dfrac 
Script to calculate the Dfrac of N2H+... 

from reproject import reproject_interp
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u

In [2]:
import numpy as np
import astropy.units as u

# Constants with astropy units
h = 6.626e-27 * u.erg * u.s  # Planck's constant
k = 1.381e-16 * u.erg / u.K  # Boltzmann's constant
c = 2.998e10 * u.cm / u.s    # Speed of light
T_bg = 2.73 * u.K            # Cosmic microwave background temperature

# Transition properties (corrected E_u)
transitions = {
    "N2H+": {
        "nu": 93.173 * u.GHz,  # GHz
        "A_ul": 3.62e-5 / u.s,  # Einstein A-coefficient
        "g_u": 3,  # Upper level degeneracy
        "E_u": (4.47 * u.K * k).to(u.erg),  # Convert temperature to energy
        "Q": 9.0  # Partition function at 10K
    },
    "N2D+": {
        "nu": 77.109 * u.GHz,  # GHz
        "A_ul": 1.2e-5 / u.s,  # Einstein A-coefficient
        "g_u": 3,
        "E_u": (3.72 * u.K * k).to(u.erg),  # Convert temperature to energy
        "Q": 7.5  # Partition function at 10K
    }
}

def J_nu(T, nu):
    """ Compute the Planck function term. """
    return (h * nu / k) / (np.exp((h * nu / (k * T)).decompose()) - 1)

def column_density(W, W_err, species="N2H+", T_ex=10 * u.K):
    """
    Calculate the column density and its uncertainty for N2H+ or N2D+.
    
    Parameters:
    - W (array or float): Integrated intensity (K km/s)
    - W_err (array or float): Error on the integrated intensity
    - species (str): "N2H+" or "N2D+"
    - T_ex (Quantity): Excitation temperature, default 10K
    
    Returns:
    - N (Quantity): Column density (cm^-2)
    - N_err (Quantity): Column density uncertainty (cm^-2)
    """
    if species not in transitions:
        raise ValueError("Invalid species. Choose 'N2H+' or 'N2D+'.")

    props = transitions[species]
    
    nu = props["nu"].to(u.Hz)  # Convert GHz to Hz
    J_ex = J_nu(T_ex, nu)
    J_bg = J_nu(T_bg, nu)
    
    factor = ((8 * np.pi * k * nu**2) / (h * c**3 * props["A_ul"] * props["g_u"])).decompose()
    exponent = np.exp((props["E_u"] / (k * T_ex)).decompose())

    # Compute column density
    N = (factor * props["Q"] * exponent * W / (J_ex - J_bg)).to(u.cm**-2)
    
    # Compute uncertainty propagation
    N_err = np.abs(N * (W_err / W))

    return N, N_err

def deuterium_fraction(N_N2D, N_N2D_err, N_N2H, N_N2H_err):
    """
    Compute the deuterium fraction map (N2D+/N2H+) and its uncertainty.

    Parameters:
    - N_N2D (array or float): Column density of N2D+
    - N_N2D_err (array or float): Uncertainty in N2D+
    - N_N2H (array or float): Column density of N2H+
    - N_N2H_err (array or float): Uncertainty in N2H+

    Returns:
    - D_frac (array or float): Deuterium fraction map
    - D_frac_err (array or float): Uncertainty in deuterium fraction
    """
    D_frac = N_N2D / N_N2H
    D_frac_err = D_frac * np.sqrt((N_N2D_err / N_N2D) ** 2 + (N_N2H_err / N_N2H) ** 2)
    return D_frac, D_frac_err

In [7]:
import numpy as np
import astropy.units as u

# Constants with astropy units
h = 6.626e-27 * u.erg * u.s  # Planck's constant
k = 1.381e-16 * u.erg / u.K  # Boltzmann's constant
c = 2.998e10 * u.cm / u.s    # Speed of light
T_bg = 2.73 * u.K            # Cosmic microwave background temperature

# Transition properties (corrected E_u)
transitions = {
    "N2H+": {
        "nu": 93.173 * u.GHz,  # GHz
        "A_ul": 3.62e-5 / u.s,  # Einstein A-coefficient
        "g_u": 3,  # Upper level degeneracy
        "E_u": (4.47 * u.K * k).to(u.erg),  # Convert temperature to energy
        "Q": 9.0  # Partition function at 10K
    },
    "N2D+": {
        "nu": 77.109 * u.GHz,  # GHz
        "A_ul": 1.2e-5 / u.s,  # Einstein A-coefficient
        "g_u": 3,
        "E_u": (3.72 * u.K * k).to(u.erg),  # Convert temperature to energy
        "Q": 7.5  # Partition function at 10K
    }
}

def J_nu(T, nu):
    """ Compute the Planck function term. """
    return (h * nu / k) / (np.exp((h * nu / (k * T)).decompose()) - 1)

def column_density(W, W_err, species="N2H+", T_ex=10 * u.K):
    """
    Calculate the column density and its uncertainty for N2H+ or N2D+.
    
    Parameters:
    - W (array or float): Integrated intensity (K km/s)
    - W_err (array or float): Error on the integrated intensity
    - species (str): "N2H+" or "N2D+"
    - T_ex (Quantity): Excitation temperature, default 10K
    
    Returns:
    - N (Quantity): Column density (cm^-2)
    - N_err (Quantity): Column density uncertainty (cm^-2)
    """
    if species not in transitions:
        raise ValueError("Invalid species. Choose 'N2H+' or 'N2D+'.")

    props = transitions[species]
    
    # Convert frequency to Hz
    nu = props["nu"].to(u.Hz)  

    # Planck radiation correction factors
    J_ex = J_nu(T_ex, nu)
    J_bg = J_nu(T_bg, nu)

    # Convert W from K km/s → K Hz
    W_cgs = W.to(u.K * u.cm / u.s)
    W_err_cgs = W_err.to(u.K * u.cm / u.s)

    # Compute the pre-factor and ensure correct units
    factor = ((8 * np.pi * k * nu**2) / (h * c**3 * props["A_ul"] * props["g_u"])).to(1 / u.cm**2)

    exponent = np.exp((props["E_u"] / (k * T_ex)).decompose())

    # Compute column density
    N = (factor * props["Q"] * exponent * W_cgs / (J_ex - J_bg)).to(u.cm**-2)
    
    # Compute uncertainty propagation
    N_err = np.abs(N * (W_err_cgs / W_cgs))

    return N, N_err

def deuterium_fraction(N_N2D, N_N2D_err, N_N2H, N_N2H_err):
    """
    Compute the deuterium fraction map (N2D+/N2H+) and its uncertainty.

    Parameters:
    - N_N2D (array or float): Column density of N2D+
    - N_N2D_err (array or float): Uncertainty in N2D+
    - N_N2H (array or float): Column density of N2H+
    - N_N2H_err (array or float): Uncertainty in N2H+

    Returns:
    - D_frac (array or float): Deuterium fraction map
    - D_frac_err (array or float): Uncertainty in deuterium fraction
    """
    D_frac = N_N2D / N_N2H
    D_frac_err = D_frac * np.sqrt((N_N2D_err / N_N2D) ** 2 + (N_N2H_err / N_N2H) ** 2)
    return D_frac, D_frac_err

# Example usage
if __name__ == "__main__":
    # Example integrated intensities and errors (replace with actual maps)
    W_N2H_plus = 2.0 * u.K * u.km / u.s
    W_N2D_plus = 0.5 * u.K * u.km / u.s
    W_err_N2H_plus = 0.1 * u.K * u.km / u.s  # Example 5% error
    W_err_N2D_plus = 0.05 * u.K * u.km / u.s  # Example 10% error

    # Compute column densities and errors
    N_N2H_plus, N_err_N2H_plus = column_density(W_N2H_plus, W_err_N2H_plus, species="N2H+")
    N_N2D_plus, N_err_N2D_plus = column_density(W_N2D_plus, W_err_N2D_plus, species="N2D+")

    # Compute deuterium fraction and its error
    D_frac, D_frac_err = deuterium_fraction(N_N2D_plus, N_err_N2D_plus, N_N2H_plus, N_err_N2H_plus)

    # Print results
    print(f"Column Density of N2H+: {N_N2H_plus:.2e} ± {N_err_N2H_plus:.2e} cm^-2")
    print(f"Column Density of N2D+: {N_N2D_plus:.2e} ± {N_err_N2D_plus:.2e} cm^-2")
    print(f"Deuterium Fraction (N2D+/N2H+): {D_frac:.2f} ± {D_frac_err:.2f}")

UnitConversionError: 's3 Hz2 / (K cm3)' and '1 / cm2' (column density) are not convertible